# Document Preprocessing

In [1]:
import os
# Setting temp dir to /srv/data directory, as it otherwise fills up the home directory too much
# Just comment out on machines that are not "Goober"
os.environ["TMPDIR"] = "/srv/data/tmp"
os.makedirs("/srv/data/tmp", exist_ok=True)

%pip install haystack-ai
%pip install nltk
%pip install markdown-it-py
%pip install mdit_plain
%pip install openai
%pip install pandas
%pip install hf_xet
%pip install ollama-haystack==2.4.2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pipelines.preprocessing_pipelines.markdown_pipeline import pipeline as markdown_pipeline
from pipelines.preprocessing_pipelines.preprocessing_pipeline import pipeline as preprocessing_pipeline
from haystack.dataclasses import ByteStream
from utils.markdown_utils import for_each_markdown_file
from tqdm import tqdm
import pandas as pd
import uuid

tqdm.pandas()

In [2]:
splitting_configs = [
    {
        "split_by": "word",
        "split_length": 48,
        "split_overlap": 10 # 20%
    },
    {
        "split_by": "word",
        "split_length": 96,
        "split_overlap": 20 # 20%
    },
    {
        "split_by": "word",
        "split_length": 384,
        "split_overlap": 80 # 20%
    },
    {
        "split_by": "word",
        "split_length": 768,
        "split_overlap": 160 # 20%
    },
    {
        "split_by": "line",
        "split_length": 1,
        "split_overlap": 0
    },
    {
        "split_by": "passage",
        "split_length": 1,
        "split_overlap": 0
    },
]

In [ ]:
from functools import partial

def preprocess_markdown(filename: str, bytes, splitting_config):
    bytestream = ByteStream(data=bytes)
    file_id = str(uuid.uuid4())
    md_documents = markdown_pipeline.run({
        "markdown_converter" : {
            "byte_streams": [bytestream],
            "id": file_id,
            "meta": {
                "title": filename
            }
        }
    })["markdown_converter"]["documents"]
    print(md_documents)
    splitter = preprocessing_pipeline.get_component("splitter")
    splitter.split_by = splitting_config["split_by"]
    splitter.split_length = splitting_config["split_length"]
    splitter.split_overlap = splitting_config["split_overlap"]
    preprocessed_documents = preprocessing_pipeline.run({
        "splitter": { "documents": md_documents }
    })["filter"]["documents"]
    data.extend([{ "document": document } for document in preprocessed_documents])

for config in splitting_configs:
    data = []
    process_with_config = partial(preprocess_markdown, splitting_config=config)
    for_each_markdown_file("data/md_files", process_with_config)
    df = pd.DataFrame(data)
    df.to_pickle(f"data/preprocessed_documents/docs_{config['split_by']}_{config['split_length']}_{config['split_overlap']}.pkl")


Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 177.30it/s]


[Document(id=2016924c-5cad-4775-93ab-7a61ff1aa29d, content: 'Im Laufe der Schwangerschaft kann das ungeborene Kind seine Position im Mutterleib mehrfach veränder...', meta: {'title': 'Äußere Wendung.md'})]


Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 183.61it/s]


[Document(id=adf49f5d-6e09-4dc0-a75b-19cff9178a83, content: 'Ein Kaiserschnitt kann aus unterschiedlichen Gründen und mit variierender Dringlichkeit notwendig we...', meta: {'title': 'Kaiserschnitt.md'})]


Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 200.98it/s]


...

[Document(id=193e7c35-9a62-4c35-8dd9-8e5ad14185c0, content: 'Die Geburt stellt einen natürlichen Ablauf dar. Durch moderne geburtshilfliche Verfahren, insbesonde...', meta: {'title': 'Medizinische Maßnahmen rund um die Geburt.md'})]


Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 132.91it/s]


[Document(id=b19719b6-f447-4f1a-975c-294c5ff7421d, content: 'Damit Sie den geplanten Eingriff schmerzfrei erleben, stehen zwei grundlegende Methoden zur Verfügun...', meta: {'title': 'Narkose.md'})]


Processing Markdown files: 100%|██████████| 6/6 [00:00<00:00, 41.71it/s]


[Document(id=12c2f1a0-5581-443a-b656-8a909d16ce48, content: 'In bestimmten Situationen wird ein vorzeitiger Geburtsbeginn medizinisch empfohlen oder notwendig:

...', meta: {'title': 'Einleitung der Geburt.md'})]


In [10]:
len(df)

21

In [6]:
document_ids = df["document"].apply(lambda doc: doc.id)
is_unique = document_ids.is_unique

print("All document IDs are unique:", is_unique)


All document IDs are unique: True


In [7]:
# Extract character length of each document
df['content_length'] = df['document'].apply(lambda x: len(x.content))

# Compute statistics
mean_length = df['content_length'].mean()
median_length = df['content_length'].median()
variance_length = df['content_length'].var()
min_length = df['content_length'].min()

print("Mean content length:", mean_length)
print("Median content length:", median_length)
print("Variance of content length:", variance_length)
print("Min content length:", min_length)

Mean content length: 317.13592233009706
Median content length: 290.0
Variance of content length: 32404.72172487707
Min content length: 15


## Splitting Evaluation

Which splitting strategy works best?
- Number of chars (with overlap)
- Number of sentences (with overlap)
- By line
- By paragraph

In [1]:
import pandas as pd

df = pd.read_pickle("data/preprocessed_documents/docs_line_1_0.pkl")

In [5]:
from utils.string_utils import normalize

# All sentence groups (flattened into one list)
all_sentences = [
    # Group 0
    "Häufig wird vorab eine Blutuntersuchung durchgeführt, Blutwerte bestimmt und ein venöser Zugang gelegt, um im Bedarfsfall Medikamente verabreichen zu können.",
    "Um vorzeitige Wehen zu vermeiden, wird häufig ein Medikament zur Wehenhemmung (Tokolyse) verabreicht.",
    "Während des gesamten Vorgangs steht Ihnen in der Regel eine Hebamme zur Seite.",
    "Falls notwendig, kann ein Bauchband angelegt werden, um die neue Lage zu stabilisieren.",
    
    # Group 3
    "Auch wenn für viele dieser Methoden keine wissenschaftlichen Nachweise zur Wirksamkeit vorliegen, berichten einige Frauen von positiven Erfahrungen.",
    "Diese Verfahren werden zumeist von Hebammen durchgeführt und erfordern oft mehrere Sitzungen.",
    "Die Kosten müssen in der Regel privat getragen werden, da sie nicht von den gesetzlichen Krankenkassen übernommen werden.",
    
    # Group 4
    "Die genaue Entscheidung hängt von der individuellen Situation ab und wird gemeinsam mit der Ärztin oder dem Arzt getroffen.",
    
    # Group 8
    "Da dabei kindliches Blut in den mütterlichen Kreislauf gelangen kann, wird meist eine schützende Impfung gegen Rhesus-Sensibilisierung verabreicht.",
    
    # Group 10
    "In manchen Fällen ist die Gabe von Sauerstoff oder sogar eine vorübergehende künstliche Beatmung erforderlich.",
    "Die wissenschaftliche Datenlage hierzu ist jedoch uneinheitlich – eindeutige Beweise für einen Zusammenhang fehlen bislang.",
    
    # Group 13
    "Diese sind in der Regel gut mit Antibiotika und kontraktionsfördernden Mitteln behandelbar.",
    "In Einzelfällen kann es zu einer schweren Blutvergiftung (Sepsis) kommen, die intensivmedizinisch behandelt werden muss.",
    "Zur Vorbeugung wird häufig ein blutgerinnungshemmendes Mittel wie Heparin verabreicht.",
    
    # Group 14
    "Die Chancen und Risiken einer geplanten Spontangeburt werden individuell mit Ihnen besprochen.",
    
    # Group 17
    "Auch hierzu werden Sie im persönlichen Gespräch informiert.",
    
    # Group 18
    "Dabei unterscheidet man:",
    
    # Group 19
    "Im Falle eines geplanten Eingriffs erhalten Sie dazu eine separate Aufklärung über Risiken und Konsequenzen.",
    
    # Group 21
    "Rund um die Uhr ist eine Anästhesistin anwesend, sodass jederzeit eine PDA gelegt werden kann, um starke Wehenschmerzen effektiv zu lindern.",
    
    # Group 25
    "Insgesamt stehen vier Familienzimmer zur Verfügung, darunter als Besonderheit das in Kooperation mit dem 1. FC Union Berlin gestaltete Themenzimmer „Eiserne Knirpsenkabine“.",
    "Insgesamt gibt es vier Familienzimmer, darunter die „Eiserne Knirpsenkabine“, ein Fan-Zimmer im Stil des 1. FC Union Berlin.",
    
    # Group 26
    "Leichte Gelbsucht wird mit Fototherapie behandelt.",
    "Sollte eine intensivere Betreuung nötig sein, erfolgt eine Zusammenarbeit mit der Neonatologie des Vivantes Klinikums Neukölln.",
    
    # Group 27
    "Die Kreißsäle sind mit allen gängigen Gebärhilfen ausgestattet, darunter eine große Gebärwanne für Wassergeburten",
    
    # Group 28
    "Es finden Stillberatungen, Rückbildungsgymnastik und Neugeborenenuntersuchungen (z. B. U2, Hörtest) statt.",
    "Danach übernimmt idealerweise eine Hebamme die Nachsorge zu Hause – sie kontrolliert Rückbildung, hilft beim Stillen und betreut das Neugeborene.",
    "Wer keine Hebamme findet, bekommt Unterstützung durch die Babylotsinnen.",
    
    # Group 29
    "Danach übernimmt idealerweise eine Hebamme die Nachsorge zu Hause – sie kontrolliert Rückbildung, hilft beim Stillen und betreut das Neugeborene.",
    "Wer keine Hebamme findet, bekommt Unterstützung durch die Babylotsinnen.",
    
    # Group 30
    "Für andere Besucher (z. B. Großeltern, Freunde) gelten feste Besuchszeiten: täglich von 15:00 bis 19:00 Uhr.",
    
    # Group 33
    "So lassen sich wichtige Werte bestimmen, insbesondere bei einem möglichen Sauerstoffmangel, und es können umgehend notwendige Schritte eingeleitet werden.",
    
    # Group 48
    "Aus diesem Grund ist es zwingend erforderlich, dass Sie sich von einer volljährigen Begleitperson nach Hause bringen lassen und dort während der ersten 24 Stunden – oder für den vom Arzt angegebenen Zeitraum – unter Aufsicht bleiben.",
    
    # Group 49
    "Diese verschwinden meist nach wenigen Stunden oder Tagen, gelegentlich halten sie Wochen oder länger an.",
    "Dauerhafte Beeinträchtigungen sind jedoch sehr selten.",
    
    # Group 52
    "In diesem Fall wird dringend zur Einleitung oder einem Kaiserschnitt geraten.",
    
    # Group 56
    "Infolgedessen kann das Neugeborene eventuell mit Antibiotika behandelt werden müssen.",
    "Eine Vereiterung der Eileiter als Folge kann zur dauerhaften Unfruchtbarkeit führen.",
    
    # Group 57
    "Informieren Sie Ihre Ärztin oder Ihren Arzt unbedingt über frühere Eingriffe!"
]

from utils.document_utils import exact_substring_match_percent

# Now check each sentence in all documents
matches = {
    sentence: any(
        exact_substring_match_percent(
            normalize(sentence),
            normalize(doc.content),
        ) >= 0.5
        for doc in df["document"]
    )
    for sentence in all_sentences
}

# Print results clearly
for sentence, found in matches.items():
    print(f"{'✅ FOUND' if found else '❌ NOT FOUND'}: {sentence}")

✅ FOUND: Häufig wird vorab eine Blutuntersuchung durchgeführt, Blutwerte bestimmt und ein venöser Zugang gelegt, um im Bedarfsfall Medikamente verabreichen zu können.
✅ FOUND: Um vorzeitige Wehen zu vermeiden, wird häufig ein Medikament zur Wehenhemmung (Tokolyse) verabreicht.
✅ FOUND: Während des gesamten Vorgangs steht Ihnen in der Regel eine Hebamme zur Seite.
✅ FOUND: Falls notwendig, kann ein Bauchband angelegt werden, um die neue Lage zu stabilisieren.
✅ FOUND: Auch wenn für viele dieser Methoden keine wissenschaftlichen Nachweise zur Wirksamkeit vorliegen, berichten einige Frauen von positiven Erfahrungen.
✅ FOUND: Diese Verfahren werden zumeist von Hebammen durchgeführt und erfordern oft mehrere Sitzungen.
✅ FOUND: Die Kosten müssen in der Regel privat getragen werden, da sie nicht von den gesetzlichen Krankenkassen übernommen werden.
✅ FOUND: Die genaue Entscheidung hängt von der individuellen Situation ab und wird gemeinsam mit der Ärztin oder dem Arzt getroffen.
✅ FOUND: Da 